In [80]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
from time import sleep
from tqdm import tqdm


creds = None
with open('../../secret.json') as creds_file:
    creds = json.load(creds_file)


# Set up the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

print('hi')
print(sp)


hi


In [81]:
def get_album_genre(album_name, artist_name):
  
    q = "artist:{} album:{}".format(artist_name, album_name)
    
    results = sp.search(q, limit=1, offset=0, type='album')

    if len(results['albums']['items']) > 0:
        artist_id = results['albums']['items'][0]['artists'][0]['id']
        album_id = results['albums']['items'][0]['id']

        # genre only availble through artist, not album through apotify api
        artist_info = sp.artist(artist_id)
        print(artist_info)
        genres = artist_info['genres']
        print(genres)
        if len(genres) > 1:
            # just taking the first one, can change if
            return genres[0]
        else:
            return None
        
    else:
        return None

In [82]:
artist_name = "Linkin Park"
song_name = "Meteora"
try:
    res = get_album_genre(song_name, artist_name)
    print(res)
except Exception as e:
    print(str(e))

alternative metal


In [83]:

cddb_df = pd.read_csv('../../cleaning_stages/4_year_repair/cddb_3.tsv', sep='\t')
tracks_df = pd.read_csv('../../cleaning_stages/4_year_repair/cddb_tracks.tsv', sep='\t')


cddb_df_trim = cddb_df[cddb_df['genre'] == 'Data']
cddb_df_trim = cddb_df_trim[['artist', 'title', 'id']]
print(cddb_df_trim)

                            artist   
2                        No Return  \
4                          Emanuel   
14                           Keedy   
25                 Jazz Signatures   
26             Bernadã¨te Bidã ude   
33            Sportfreunde Stiller   
34         Rennie Pilgrim Presents   
38                            Deux   
40                         Various   
41                 Matsuka Phonics   
43                        Robotika   
48                  Rolling Stones   
49    Ã¤ã®ã«ã¥ã°ã¤ Ã¹ã¬ Ã¤ã¢ã¸ã¥ã¡   
61           Â¿â©ã‡ã _â½âºã„ã‰ã„â¡   
62                    Diversetion)   
9332                    Pink Floyd   

                                                  title      id  
2                                       Self Mutilation  100002  
4                                            Felicidade  100004  
14                                     Chase The Clouds  100015  
25                                      Jazz Signatures  100028  
26                     

In [84]:
# lists are much faster than dataframes, converting
cddb_list = cddb_df_trim.values.tolist()
tracks_list = tracks_df.values.tolist()

print(f'num null years: ${len(cddb_list)}')

num null years: $16


In [91]:
def start_repair(cd_list):
    
    total_rows = len(cd_list)
    for cddb_row in tqdm(cd_list, total=total_rows, desc='Processing rows', ncols=100):
    #for cddb_row in cd_list:
  
        artist_name = cddb_row[0]
        album_name = cddb_row[1]

        fix = get_album_genre(album_name, artist_name)
        if fix:
            # update cddb_df directly
            idx_list = cddb_df.index[cddb_df['id'] == cddb_row[2]].tolist()
            idx = idx_list[0]
            cddb_df.at[idx,'category'] = fix
            cddb_df.at[idx,'genre'] = fix
            print("fixed - :" + str(cddb_row) + "idx: " + str(idx))
    

In [92]:
start_repair(cddb_list)

Processing rows:  38%|█████████████████▋                             | 6/16 [00:00<00:01,  8.41it/s]

fixed - :['Sportfreunde Stiller', 'Die Gute Seite', 100037]idx: 33


Processing rows: 100%|██████████████████████████████████████████████| 16/16 [00:01<00:00,  8.58it/s]


In [93]:
#save to output
cddb_df.to_csv('../../cleaning_stages/5_genre_repair/cddb_4.tsv', sep="\t", index=False)
tracks_df.to_csv('../../cleaning_stages/5_genre_repair/cddb_tracks.tsv', sep="\t", index=False)